#DATASCI W261: Machine Learning at Scale

**Nick Hamlin** (nickhamlin@gmail.com)  
**Tigi Thomas** (tgthomas@berkeley.edu)  
**Rock Baek** (rockb1017@gmail.com)  
**Hussein Danish** (husseindanish@gmail.com)  
  
Time of Submission: 10:30 AM EST, Saturday, Feb 27, 2016  
W261-3, Spring 2016  
Week 7 Homework

###Submission Notes:
- For each problem, we've included a summary of the question as posed in the instructions.  In many cases, we have not included the full text to keep the final submission as uncluttered as possible.  For reference, we've included a link to the original instructions in the "Useful Reference" below.
- Some aspects of this notebook don't always render nicely into PDF form.  In these situations, please reference [the complete rendered notebook on Github](https://github.com/nickhamlin/mids_261_homework/blob/master/HW7/MIDS-W261-2015-HWK-Week07-Hamlin-Thomas-Baek-Danish.ipynb)


###Useful References:
- **[Original Assignment Instructions](https://www.dropbox.com/s/26ejqhkzqdidzwj/HW7-Questions.txt?dl=0)**


In [ ]:
#Use this to make sure we reload the MrJob code when we make changes
%load_ext autoreload
%autoreload 2
#Render matplotlib charts in notebook
%matplotlib inline

#Import some modules we know we'll use frequently
import numpy as np
import pylab as plt

## HW 7.0

### HW 7.0 - Problem Statement
In this part of your assignment you will develop the base of your code for the week.

Write MRJob classes to find shortest path graph distances, 
as described in the lectures. In addition to finding the distances, 
your code should also output a distance-minimizing path between the source and target.
Work locally for this part of the assignment, and use 
both of the undirected and directed toy networks.

To proof you code's function, run the following jobs

- shortest path in the undirected network from node 1 to node 4
Solution: 1,5,4 

- shortest path in the directed network from node 1 to node 5
Solution: 1,2,4,5

and report your output---make sure it is correct!

### HW 7.0 - MR job definition
For brevity, we've combined the initialization step and the main job into a single unit of code.  During the first pass, we check to make sure each node has been properly initialized and, if not, initialize it.  

In [1]:
%%writefile MRbfs.py
from __future__ import division
from mrjob.job import MRJob
from mrjob.job import MRStep
from mrjob.step import MRJobStep
import re
import ast

WORD_RE = re.compile(r"[\w']+")
 
class MRbfs(MRJob):
    
    def __init__(self, *args, **kwargs):
        super(MRbfs, self).__init__(*args, **kwargs)
        self.sN = ''
        self.dN = ''
        
    def mapper_init(self):
        #load the begin and End Nodes.. 
        with open ('start_end.txt', "rU") as f:
            for line in f:
                self.sN, self.dN = line.split(',')
        
    def mapper(self, _, line):
        line = line.strip('\n')
        data = line.split("\t")
        status = 'U' #everything is unvisited initially
        
        # This is the first inialization (from original graph file)
        # there are only 2 elements, the node and adjacency list
        if(len(data) == 2): 
            nid = data[0]
            N = ast.literal_eval(data[1])
            
            # if our node is the start node , initialize the start
            # distance at source = 0.0
            if nid == self.sN:
                ds = 0.0
                status = 'V'
            else: 
                # if this is not the start node, intialize to inf
                ds = float("inf")  
                
            # yeild the root node , dist and graph structure...
            # we need this to eventually for next iteration..
            yield  nid, (ds, N, status, None)
            
            # nor each of the nodes in the adjacency list, 
            # we expand frontier with starting distance.
            for m,d in N.iteritems():
                #new distance is going to be from root node  + dist              
                newdist = d+ds
                
                #not sure if this is needed really..just 
                #marking this single node , dist as in the Q.
                if newdist < float("inf"):
                    status = 'Q'
                    
                yield m, (newdist, None, status, nid) 
                
        # from Iteration 1 onwards we'll land here...more data items to track
        elif len(data) == 5:  
            nid = data[0]   #cur Node
            dist = data[1]  #cur dist
            N = ast.literal_eval(data[2]) # adj list - graph to use for next iter
            status = data[3] #status - U, Q, V
            visited = data[4] #Pre vist before this node..
            
            #debug
            #print nid, dist, N,status, visited
            
            # just redo dist, so it is summable ? is this needed ? 
            dist = float("inf") if dist == 'inf' else float(dist)
              
            # yeild the root node , dist and graph structure...
            # we need this to eventually for next iteration..
            yield  nid, (dist, N, status, visited)
            
            # expand the adjacency list frontier..
            if N != None:
                for m,d in N.iteritems():
                    newdist = d+dist
                    if newdist < float("inf") and status <> 'V':
                        status = 'Q'
                    yield m, (d+dist, None, status, nid)     

    #just to debug and see the output coming into the reducer.. 
    def debug_reducer(self, node, distances):
        for dist in distances:
            yield node, dist
            
    def reducer(self, node, distances):
        adjList = None
        sdist = float("inf")
        visited = ''
        status = 'U'
        for dist in distances:
            # for the non-root nodes will have None instead of
            # adjacency list
            if(dist[1] != None):
                adjList = dist[1]           
            if dist[0] < sdist:
                sdist = dist[0]
                visited = dist[3]
       
        # if we got a distance that is not 'inf' that means
        # we vistied the node..mark as V
        if sdist < float("inf"):
            status = 'V'
        yield node, (sdist, adjList, status, visited)
        
    def steps(self):
        return [MRStep( mapper_init=self.mapper_init, 
                        mapper=self.mapper
                        #,reducer=self.debug_reducer
                        ,reducer=self.reducer
                        #,jobconf = {
                        #      'mapred.map.tasks':28,
                        #      'mapred.reduce.tasks':28
                        #    }
                      )
                #,MRStep(reducer=self.reducer
                #       ,jobconf = {
                #               'mapred.map.tasks':10
                #              ,'mapred.reduce.tasks':1
                #              ,'stream.num.map.output.key.fields':2 
                #              ,'mapred.output.key.comparator.class': 'org.apache.hadoop.mapred.lib.KeyFieldBasedComparator'
                #              ,'mapred.text.key.comparator.options': '-k2,2rn'                             
                #            }
                #      )
                ]
if __name__ == '__main__':
    MRbfs.run()

Writing MRbfs.py


### HW 7.0 - Test BFS on multiple toy data sets

For each of the Data Sets, 
- create testgraph.txt and start_end.txt
- Run smoke test for first iteration
- Rum the Driver
   


#### Undirected Toy Test Set 1 - create graph file and associated start-end node file

In [2]:
%%writefile testgraph.txt
A	{'B':10, 'D':5}
B	{'C':1, 'D':2}
C	{'E':4}
D	{'B':3, 'C':9, 'E':2}
E	{'A':7, 'C':6}

Writing testgraph.txt


In [3]:
%%writefile start_end.txt
A,E

Writing start_end.txt


#### Directed Toy Test Set 2 - create graph file and associated start-end node file

In [4]:
%%writefile testgraph.txt
1	{'2':1, '6':1}
2	{'1':1, '3':1, '4':1}
3	{'2':1, '4':1}
4	{'2':1, '5':1}
5	{'1':1, '2':1, '4':1}

Overwriting testgraph.txt


In [5]:
%%writefile start_end.txt
1,5

Overwriting start_end.txt


#### Undirected Toy Test - create graph file and associated start-end node file

In [6]:
%%writefile testgraph.txt
1	{'2': 1,'5': 1}
2	{'1': 1,'3': 1,'4': 1,'5': 1}
3	{'2': 1, '4': 1}
4	{'2': 1,'3': 1,'5': 1}
5	{'1': 1, '2': 1, '4': 1}

Overwriting testgraph.txt


In [7]:
%%writefile start_end.txt
1,4

Overwriting start_end.txt


#### Just a quick test run with first Iteration output. 

In [9]:
%reload_ext autoreload
%autoreload 2
from MRbfs import MRbfs
from __future__ import division

mr_job = MRbfs(args=['testgraph.txt', '--file=start_end.txt','--no-strict-protocols'])
with mr_job.make_runner() as runner: 
    runner.run()
    for line in runner.stream_output():
        print mr_job.parse_output_line(line)
            

('1', [0.0, {'2': 1, '5': 1}, 'V', None])
('2', [1.0, {'1': 1, '3': 1, '5': 1, '4': 1}, 'V', '1'])
('3', [inf, {'2': 1, '4': 1}, 'U', u''])
('4', [inf, {'3': 1, '2': 1, '5': 1}, 'U', u''])
('5', [1.0, {'1': 1, '2': 1, '4': 1}, 'V', '1'])


#### Now try running the actual job!

In [10]:
%reload_ext autoreload
%autoreload 2
from __future__ import division
from MRbfs import MRbfs

mr_job = MRbfs(args=['testgraph.txt', '--file=start_end.txt','--no-strict-protocols'])

iterate = 1
stop = False

# using this to total the overall distance so far.. 
# another stop condition if we have no new shorted
# distances.
last_total_dist = float("inf")

#Keep our start and end nodes.
start_node = ''
end_node = ''

# we are going to track the distance to the end
# node. If we are getting the same distance for the last
# few iterations we stop.
end_sdist = float("inf")

#just for some printng stuff
path = {}

#Read our Start and End node values.
#We keep this to determine stop conditions etc.
with open ('start_end.txt', "rU") as f:
    for line in f:
        start_node, end_node = line.split(',')
        prev_visited = start_node

        
# We are iterating till we stop or we hit an interation
# count. Count just becuase we want to stop incase there 
# is a run-off into an inf look.
while(not stop and iterate <= 10):
    
    #kick off our mrjob
    with mr_job.make_runner() as runner: 
        runner.run()
        
        # some pretty print for readibility, debugging etc...
        print ""
        print "#" * 60
        print "Iteration : {0}".format(iterate)
        with open("testgraph.txt", 'w+') as f:
            
            this_iter_total_dist  = 0
            for line in runner.stream_output():
                nid,distances =  mr_job.parse_output_line(line)
                dist,adjlist,status,visited  = distances
                
                # get the real distance in float so we can total it.. 
                dist = float("inf") if dist == 'inf' else float(dist)
                
                # check if for the last node we have hit a non-inf distance
                # and there are no new shorter distances.. then stop..
                if(nid == end_node and dist < end_sdist):
                    end_sdist = dist
                elif(nid == end_node and dist == end_sdist and dist != float("inf")):
                    #found the shortest distance to the end node.. We can stop..
                    stop = True
                
                # summing the total distance of all nodes in this iteration
                # just so we can stop if overall no change in distance..
                this_iter_total_dist += dist
                print "{}\t{}\t{}\t{}\t{}".format(nid,dist,adjlist,status,visited)
                f.write("{}\t{}\t{}\t{}\t{}\n".format(nid,dist,adjlist,status,visited))
                
                # add the path to our path collection..
                # for printing shortes path etc.. 
                # node , (visted from, distance)
                path[str(nid)] = (str(visited), str(dist))
            
            print ""
            print "Prev Total: {0},  Cur Total: {1}".format(last_total_dist, this_iter_total_dist)
             
            # DEBUG the current iteration path list traversal.
            #for node, dist in path.iteritems():
            #    print node, dist

            # just traversing our path from end node, following the visited before chain
            # the dict is in the form: node , (visted from, distance)
            print ""
            node = end_node
            sdist = path[node][1] 
            count = 0
            spath = []
            if( path[node][0] != ''):  
                while( node != start_node and node != 'None' and count <= 10):
                    print node, path[node][0], path[node][1]
                    spath.append(node)
                    node = path[node][0]
                spath.append(node)
                print node, path[node][0], path[node][1]
            spath.reverse()    
            print "Shortest Path: {}".format(spath)
            print "Shortest Distance: {}".format(sdist)  
            
            # here we are summing the totals - another stop condition.
            if this_iter_total_dist < last_total_dist or this_iter_total_dist == float("inf"):
                last_total_dist = this_iter_total_dist
            elif this_iter_total_dist == last_total_dist:
                last_total_dist = this_iter_total_dist
                stop = True
                
        #Increasea our final block if we get into inf loop.
        iterate += 1   
            
print ""
print "Done!"


############################################################
Iteration : 1
1	0.0	{'2': 1, '5': 1}	V	None
2	1.0	{'1': 1, '3': 1, '5': 1, '4': 1}	V	1
3	inf	{'2': 1, '4': 1}	U	
4	inf	{'3': 1, '2': 1, '5': 1}	U	
5	1.0	{'1': 1, '2': 1, '4': 1}	V	1

Prev Total: inf,  Cur Total: inf

Shortest Path: []
Shortest Distance: inf

############################################################
Iteration : 2
1	0.0	{'2': 1, '5': 1}	V	None
2	1.0	{'1': 1, '3': 1, '5': 1, '4': 1}	V	1
3	2.0	{'2': 1, '4': 1}	V	2
4	2.0	{'3': 1, '2': 1, '5': 1}	V	2
5	1.0	{'1': 1, '2': 1, '4': 1}	V	1

Prev Total: inf,  Cur Total: 6.0

4 2 2.0
2 1 1.0
1 None 0.0
Shortest Path: ['1', '2', '4']
Shortest Distance: 2.0

############################################################
Iteration : 3
1	0.0	{'2': 1, '5': 1}	V	None
2	1.0	{'1': 1, '3': 1, '5': 1, '4': 1}	V	1
3	2.0	{'2': 1, '4': 1}	V	2
4	2.0	{'3': 1, '2': 1, '5': 1}	V	2
5	1.0	{'1': 1, '2': 1, '4': 1}	V	1

Prev Total: 6.0,  Cur Total: 6.0

4 2 2.0
2 1 1.0
1 None 0.0
Shortest P

##HW7.1 

### HW 7.1 Problem Statement

Using MRJob, explore the synonyms network data.
Consider plotting the degree distribution (does it follow a power law?),
and determine some of the key features, like:

number of nodes, 
number links,
or the average degree (i.e., the average number of links per node),
etc...

As you develop your code, please be sure to run it locally first (though on the whole dataset). 
Once you have gotten you code to run locally, deploy it on AWS as a systems test
in preparation for our next dataset (which will require AWS).

In [12]:
%%writefile mrexplorenltk.py

from mrjob.job import MRJob
from mrjob.job import MRStep
import csv
import heapq
from operator import itemgetter
import re
import ast

class mrexplorenltk(MRJob):
    custom_jobconf = None
    def __init__(self, *args, **kwargs):
        super(mrexplorenltk, self).__init__(*args, **kwargs)
        self.node_count = 0
        self.link_count = 0
            
    def mapper(self, _, line):
        line = line.strip('\n')
        data = line.split("\t")
        nid = data[0]
        N = ast.literal_eval(data[1])
        self.node_count += 1
        self.link_count += len(N)
     
    def mapper_final(self):
        yield 'Total Node_Count', self.node_count
        yield 'Total Link_Count', self.link_count
        
    def combiner(self, item, counts):
        yield item, sum(counts)
        
    def reducer(self, item, counts):
        yield item, sum(counts)
    
    def steps(self):
        return [MRStep(  mapper=self.mapper
                    ,mapper_final = self.mapper_final
                    ,combiner=self.combiner
                    ,reducer=self.reducer
                    #,jobconf = {
                    #      'mapred.map.tasks':28,
                    #      'mapred.reduce.tasks':28
                    #    }
                )
            #,MRStep(reducer=self.reducer
            #       ,jobconf = {
            #               'mapred.map.tasks':10
            #              ,'mapred.reduce.tasks':1
            #              ,'stream.num.map.output.key.fields':2 
            #              ,'mapred.output.key.comparator.class': 'org.apache.hadoop.mapred.lib.KeyFieldBasedComparator'
            #              ,'mapred.text.key.comparator.options': '-k2,2rn'                             
            #            }
            #      )
            ]
                
if __name__ == '__main__':
    mrexplorenltk.run()

Writing mrexplorenltk.py


In [16]:
!cat ./Data/synNet/synNet.txt > synNet.txt

In [14]:
%reload_ext autoreload
%autoreload 2

from mrexplorenltk import mrexplorenltk

mr_job = mrexplorenltk(args=['synNet.txt','--no-strict-protocols'])
with mr_job.make_runner() as runner:
    runner.run()
    count = 0
    for line in runner.stream_output():
            name,value =  mr_job.parse_output_line(line)
            print mr_job.parse_output_line(line)
         

('Total Link_Count', 61134)
('Total Node_Count', 8271)


# TODO - Avg etc.. for 7.1

##HW7.2

### HW 7.2 Problem Statement

Write (reuse your code from 7.0) an MRJob class to find shortest path graph distances, 
and apply it to the NLTK synonyms network dataset. 

Proof your code's function by running the job:

- shortest path starting at "walk" (index=7827) and ending at "make" (index=536),

and showing you code's output. Once again, your output should include the path and the distance.

As you develop your code, please be sure to run it locally first (though on the whole dataset). 
Once you have gotten you code to run locally, deploy it on AWS as a systems test
in preparation for our next dataset (which will require AWS).

#### Copy synNet.txt for run , since after the run synNet.txt will change..

In [24]:
!cat ./Data/synNet/synNet.txt > synNet.txt

#### Specify Start and End Nodes

In [15]:
%%writefile start_end.txt
7827,536

Overwriting start_end.txt


#### Run Driver (Comment out debug print statements, since we'd have to many items to list..)

In [16]:
%reload_ext autoreload
%autoreload 2
from MRbfs import MRbfs
from __future__ import division

mr_job = MRbfs(args=['synNet.txt', '--file=start_end.txt','--no-strict-protocols'])
#mr_job = MRbfs(args=['-r', 'emr', 'synNet.txt', '--file=start_end.txt'])

iterate = 1
stop = False
last_total_dist = float("inf")
start_node = ''
end_node = ''
end_sdist = float("inf")
path = {}
with open ('start_end.txt', "rU") as f:
    for line in f:
        start_node, end_node = line.split(',')
        prev_visited = start_node

while(not stop and iterate <= 20):
    with mr_job.make_runner() as runner: 
        runner.run()
        print ""
        #print "#" * 60
        print "Iteration : {0}".format(iterate)
        with open("synNet.txt", 'w+') as f:
            this_iter_total_dist  = 0
            for line in runner.stream_output():
                nid,distances =  mr_job.parse_output_line(line)
                dist,adjlist,status,visited  = distances
                if (dist == 'inf'):
                    dist = float('inf')
                else:
                    dist = float(dist)
                
                if(nid == end_node and dist < end_sdist):
                    end_sdist = dist
                elif(nid == end_node and dist == end_sdist and dist != float("inf")):
                    #found the shortest distance to the end node.. We can stop..
                    stop = True
                
                this_iter_total_dist += dist
                #print "{}\t{}\t{}\t{}\t{}".format(nid,dist,adjlist,status,visited)
                f.write("{}\t{}\t{}\t{}\t{}\n".format(nid,dist,adjlist,status,visited))
                
                path[str(nid)] = (str(visited), str(dist))
            
            #print ""
            #print "Prev Total: {0},  Cur Total: {1}".format(last_total_dist, this_iter_total_dist)
                        
            #for node, dist in path.iteritems():
            #    print node, dist

            print ""
            node = end_node
            sdist = path[node][1] 
            count = 0
            spath = []
            if( path[node][0] != ''):  
                while( node != start_node and node != 'None' and count <= 10):
                    #print node, path[node][0], path[node][1]
                    spath.append(node)
                    node = path[node][0]
                spath.append(node)
                #print node, path[node][0], path[node][1]
            spath.reverse()    
            print "Shortest Path: {}".format(spath)
            print "Shortest Distance: {}".format(sdist)  
            
            if this_iter_total_dist < last_total_dist or this_iter_total_dist == float("inf"):
                last_total_dist = this_iter_total_dist
            elif this_iter_total_dist == last_total_dist:
                last_total_dist = this_iter_total_dist
                stop = 1
        #just in case this thing does not stop..
        iterate += 1   

print ""
print "Done!"


Iteration : 1

Shortest Path: []
Shortest Distance: inf

Iteration : 2

Shortest Path: []
Shortest Distance: inf

Iteration : 3

Shortest Path: ['7827', '1426', '1668', '536']
Shortest Distance: 3.0

Iteration : 4

Shortest Path: ['7827', '1426', '1668', '536']
Shortest Distance: 3.0

Done!


##HW7.3 

### HW 7.3 Problem Statement
Using MRJob, explore the Wikipedia network data on the AWS cloud. Reuse your code from HW 7.1---does is scale well? 
Be cautioned that Wikipedia is a directed network, where links are not symmetric. 
So, even though a node may be linked to, it will not appear as a primary record itself if it has no out-links. 
This means that you may have to ADJUST your code (depending on its design). 
To be sure of your code's functionality in this context, run a systems test on the directed_toy.txt network.

##HW 7.4

### HW 7.4 - Problem Statement

Using MRJob, find shortest path graph distances in the Wikipedia network on the AWS cloud.
Reuse your code from 7.2, but once again be warned of Wikipedia being a directed network.
To be sure of your code's functionality in this context, run a systems test on the directed_toy.txt network.

When running your code on the Wikipedia network, proof its function by running the job:

- shortest path from "Ireland" (index=6176135) to "University of California, Berkeley" (index=13466359),

and show your code's output.

Once your code is running, find some other shortest paths and report your results.

##HW 7.5

### HW 7.5 Problem Statement
Suppose you wanted to find the largest network distance from a single source,
i.e., a node that is the furthest (but still reachable) from a single source.

How would you implement this task? 
How is this different from finding the shortest path graph distances?

Is this task more difficult to implement than the shortest path distance?

As you respond, please comment on program structure, runtimes, iterations, general system requirements, etc...

##End of Submission